In [9]:
import sys
sys.path
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import time
import os
import re
DEBUG = 1
from datetime import datetime, timedelta
import dateparser
import calendar

In [18]:
url = 'https://www.quora.com/q/expresshowyoufeel/When-you-looked-at-the-mirror-today-what-did-you-see'

In [39]:
chromedriver = "/home/youcef/Documents/chromedriver"
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.get(url)

WebDriverException: Message: 'chromedriver' executable may have wrong permissions. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [28]:
# Extract Question
question_title = driver.find_element_by_xpath('.//span[contains(@class, "rendered_qtext")]').text
print(question_title)

Express How You Feel


In [29]:
# Extract answer count
answer_count = driver.find_element_by_xpath("//div[@class = 'answer_count']")
text_count = answer_count.text.split()
count = int((text_count[0].split('+'))[0])
print(count)

47


In [30]:
# Extract Topics
topic_list = []
topics = driver.find_elements_by_xpath(".//span[@class = 'TopicName TopicNameSpan']")
for topic in topics:
    topic_list.append(topic.text)
print(topic_list)

[]


In [31]:
# Scroll down page
current_scroll_position, new_height= 0, 1
speed = 10
while current_scroll_position <= new_height:
    current_scroll_position += speed
    driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
    new_height = driver.execute_script("return document.body.scrollHeight")

In [32]:
# Expand all the answer
hidden_list = driver.find_elements_by_xpath(".//div[@class='ui_qtext_truncated ui_qtext_truncated_regular']/div[@class='ui_qtext_truncated_text']")
#print('Number of hidden answer:',len(hidden_list))
for hidden in hidden_list:
    hidden = hidden.find_element_by_xpath(".//a[@class='ui_qtext_more_link']")
    webdriver.ActionChains(driver).move_to_element(hidden).click(hidden).perform()

In [33]:
# Extract each respond
# Name, Date, content, no of upvote, view
check_ans_count = 0

name_list = []
upvote_list = []
ans_list = []
date_list = []
view_list = []
    
respone_list = driver.find_elements_by_xpath(".//div[contains(@class, 'AnswerBase')]")
for respone in respone_list:
    # Name
    try:
        user_name = respone.find_element_by_xpath('.//a[@class = "user"]')
        print(user_name.text)
        name_list.append(user_name.text)

    except:
        user_name = respone.find_element_by_xpath('.//span[contains(@class, "anon_user")]')
        #print(user_name.text)
        name_list.append(user_name.text)

    # Date
    date = respone.find_element_by_xpath(".//a[@class = 'answer_permalink']")
    date_split = date.text.split('Answered ')
    question_date = date_split[1]
    date_list.append(question_date)
    #print(question_date)

    # Features that show after expand
    try:
        expanded_answer = respone.find_element_by_xpath(".//div[@class = 'ExpandedContent ExpandedAnswer']")
    except:
        expanded_answer = respone.find_element_by_xpath(".//div[@class = 'ExpandedAnswer ExpandedContent']")

    # Answer
    answer = expanded_answer.find_element_by_xpath(".//div[@class='ui_qtext_expanded']/span[@class='ui_qtext_rendered_qtext']")
    #print(answer.text)
    ans_list.append(answer.text)

    # Answer upvote
    try:
        upvote = expanded_answer.find_element_by_xpath(".//a[@class='VoterListModalLink']")
        res = [int(i) for i in upvote.text.split() if i.isdigit()] 
        #print('Number of upvote:', res[0])
        upvote_list.append(res[0])
    except:
        #print('Number of upvote: 0')
        upvote_list.append(0)

    # Answer view
    answer_footer = respone.find_element_by_xpath(".//div[contains(@class, 'AnswerFooter')]")
    view = int((answer_footer.text.split())[0])
    view_list.append(view)
    print(view)

    print()    
    check_ans_count += 1

print("Respond collected:", check_ans_count)

KIm Hoskins


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class = 'ExpandedAnswer ExpandedContent']"}
  (Session info: headless chrome=81.0.4044.92)


In [ ]:
# More statistic in another page
driver.get(url+'/log')

In [ ]:
# Scroll down page -- make it faster later
current_scroll_position, new_height= 0, 1
speed = 5
while current_scroll_position <= new_height:
    current_scroll_position += speed
    driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
    new_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
# Get the last item
last_item = driver.find_elements_by_xpath(".//div[@class = 'pagedlist_item']")[-1]

In [ ]:
# Extract question anonymity
try:
    user_name = last_item.find_element_by_xpath(".//span[@class = 'anon_user']").text
except:
    user_name = last_item.find_element_by_xpath(".//a[@class = 'user']").text

print(user_name)

In [ ]:
# Extract create time
question_data = last_item.find_element_by_xpath(".//p[@class = 'log_action_bar']")
question_data_split = question_data.text.split('· ')
question_date = question_data_split[1]
print(question_date)

In [ ]:
# Question Stats
# Followers, Views
stats = driver.find_elements_by_xpath(".//div[@class = 'u-flex u-flex-align--center u-padding-vert--xs u-text--gray-light pass_color_to_child_links u-sans-font-main--small']")
no_of_follower = int((stats[0].text.split(' '))[0])
print(no_of_follower)

no_of_view = (stats[1].text.split(' '))[0]
print(no_of_view)